## Libraries

In [1]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2
import copy
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'

## Definitions

In [7]:
def get_scale(img, recursive_deepness = 0):
    ##if metadata not aviable
    img_original = copy.deepcopy(img)

    off_x = 0
    off_y = 0

    if img.shape[0] > 1700:
        font_scale = 4

        off_y = 1770
        off_x = 1110

        foot = img[1770:1820, 1110:-10,:]
    else:
        font_scale = 1
        off_y = 885
        off_x = 655-recursive_deepness
        foot = img[885:914, 655-recursive_deepness:-20+recursive_deepness,:]


    text = pytesseract.image_to_string(foot)

    text = numbers_only = ''.join([char for char in text if char.isdigit()])
 
    if int(text) == 200 or  int(text) == 500:
        apeture_size = 3
    if int(text) == 100:
        apeture_size = 5

    apeture_size = 5
    
    # Use canny edge detection
    edges = cv2.Canny(foot,50,255,apertureSize=apeture_size)

    # Apply HoughLinesP method to 
    # to directly obtain line end points
    lines_list =[]
    lines = cv2.HoughLinesP(
                edges, # Input edge image
                1, # Distance resolution in pixels
                np.pi/180, # Angle resolution in radians
                threshold=100, # Min number of votes for valid line
                minLineLength=10, # Min allowed length of line
                maxLineGap=15 # Max allowed gap between line for joining them
                )
    
    cdstP = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    if lines is not None:
        for i in range(0, len(lines)):
            l = lines[i][0]
            cv2.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

    x_max = 0
    y_max = 0
    x_min = 0
    y_min = 0

    length = 0
    index = 0
    
    # Iterate over points
    for points in lines:
        # Extracted points nested in the list
        x1,y1,x2,y2=points[0]

        if index == 0:
            x_min = x1
            y_min = y1
            x_max = x2
            y_max = y2
            index+=1
        if(x1<x_min):
            x_min = x1 
            y_min = y1
            
        if(x2>x_max):
            x_max = x2 
            y_max = y2
        
        # Draw the lines joing the points
        # On the original image

    length = x_max - x_min
    
    x1 = x_min+off_x
    y1 = y_min+off_y
    x2 = x_max+off_x
    y2 = y_max+off_y

    if  foot.shape[1] - length <= 8 :
        window_incr = recursive_deepness +2
        length, x1, y1, x2, y2,text =  get_scale(img_original, recursive_deepness = window_incr )

    return length, x1, y1, x2, y2,text

In [17]:

 
# Read image
img = cv2.imread('./Isolated/NP sample 1_001.tif'  )



img_original = copy.deepcopy(img)
length, x1, y1, x2, y2,text = get_scale(img)
cv2.line(img_original,(x1,y1),(x2,y1),(0,255,0),2)

cv2.putText(img_original, f"{np.round(length,2)}"+" px", (x1,y1+20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 

# Save the result image
cv2.imwrite('detectedLines.png',img_original)

img = cv2.imread('./Isolated/NP sample 1_024.tif'  )

img_original = copy.deepcopy(img)
length, x1, y1, x2, y2,text = get_scale(img)
cv2.line(img_original,(x1,y1),(x2,y1),(0,255,0),2)

cv2.putText(img_original, f"{np.round(length,2)}"+" px", (x1,y1+20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 

# Save the result image
cv2.imwrite('detectedLines2.png',img_original)

True

In [5]:
def extract_circles(img, name):

    img_original = copy.deepcopy(img)

    if img.shape[0] > 1700:
        font_scale = 4

        img = img[:1770, :,:]
    else:
        font_scale = 1
        img = img[:882, :,:]

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find edges using the Canny edge detector (optional)
    _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)



    # Define the kernel (structuring element) for erosion
    kernel = np.ones((3, 3), np.uint8)  # Adjust the size as needed

    # Apply erosion to the image
    eroded_image = cv2.erode(thresholded, kernel, iterations=5)  # You can change the number of iterations

    #eroded_image = cv2.dilate(eroded_image, kernel, iterations=2) 



    # Find contours in the image
    contours, _ = cv2.findContours(eroded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize variables to store the center and radius of the biggest inscribed circle
    biggest_radius = 0
    biggest_center = (0, 0)

    radius_list=np.zeros((2,len(contours)))

    # Iterate through the detected contours
    idx =0
    length, x1, y1, x2, y2, text = get_scale(img_original)
    cv2.line(img_original,(x1,y1),(x2,y1),(0,255,0),2)
    cv2.putText(img_original, f"{np.round(length,2)}"+" px", (x1,y1+20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 

    
    for contour in contours:

        mask = np.zeros_like(img, dtype=np.uint8)
        mask = cv2.drawContours(mask, [contour], 0, 255, thickness=cv2.FILLED)
        mask = mask.astype(np.uint8)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        mask = cv2.dilate(mask, kernel, iterations=8) 
        # Distance Transform
        dt = cv2.distanceTransform(mask, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)

        # Find max value
        _, radius, _, center = cv2.minMaxLoc(dt)

        # Fit a circle around the contour
        mask = cv2.dilate(mask, kernel, iterations=2) 
        contours_big, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        (x, y), radius2 = cv2.minEnclosingCircle(contours_big[0])

        # Check if the current circle is the largest found so far
        
        largest_radius = radius2
        largest_center = (int(x), int(y))

        if np.abs(radius2 - radius) < 6:
            continue

        # Output image
        cv2.circle(img_original, tuple(center), int(radius), (255, 0, 0), 2)
        cv2.putText(img_original, f"{np.round(radius,3)}", tuple(int(c+radius) for c in center),
            cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 0, 0), 2) 
        
        cv2.circle(img_original, largest_center, int(largest_radius), (0, 255, 0), 2)
        cv2.putText(img_original, f"{np.round(largest_radius,2)}", tuple(int(c-radius) for c in largest_center),
            cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), 2) 


        
        radius_list[0,idx] = radius
        radius_list[1,idx] = largest_radius
        idx+=1

    return radius_list,length, text

In [8]:
def write_csv(line, csv_file_path = 'example.csv' ):


    # Open the CSV file in write mode
    with open(csv_file_path, 'w', newline='') as csv_file:
        # Create a CSV writer object
        csv_writer = csv.writer(csv_file)

        # Write the data to the CSV file
        csv_writer.writerows(line)

    print(f'CSV file "{csv_file_path}" created successfully.')

## Main Script

In [ ]:
# Specify the path to the folder containing your images
folder_path = "./"

# Get a list of all files in the folder
folders = os.listdir(folder_path)

for folder in folders:

    # Construct the full path to the image file
    files_path = os.path.join(folder_path, folder)
    
    if not os.path.isdir(files_path):
        continue

    files = os.listdir(files_path)

    # Filter only image files (you can customize this based on your image extensions)
    image_files = [f for f in files if f.lower().endswith(('.tif'))]

    data_cv = []

    data_cv.append(["Name", "Inner radius (px)","Outer radius (px)","Scale (px)","Scale (nm)"])

    # Loop through each image file and open it using OpenCV
    for image_file in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(files_path, image_file)

        # Read the image using OpenCV
        img = cv2.imread(image_path)
        
        print(image_path)
        print(img.shape)

        isExist = os.path.exists('Output')
        if not isExist:
        # Create a new directory because it does not exist
            os.makedirs('Output')
            
        new_folder = os.path.join('Output/'+folder)
        isExist = os.path.exists(new_folder)
        if not isExist:
        # Create a new directory because it does not exist
            os.makedirs(new_folder)

        
    

        image_file_name = os.path.join('Output/'+folder, image_file)

        # Extract the circles

        radius,length, meters= extract_circles(img, image_file_name)
        try:
            # Extract the circles
            radius,length, meters= extract_circles(img, image_file)
            for i in range(radius.shape[1]):
                if radius[0,i] == 0.0 and radius[1,i] == 0.0:
                    continue
                data_cv.append([image_file, radius[0,i],radius[1,i],length,meters])


        except:

            print('error '+image_file)
        
    file_name = os.path.join('Output/'+folder)
    write_csv(data_cv, file_name+'/out.csv' )